In [5]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import keras
from keras.layers import convolutional, Dense, Activation,pooling
from keras.layers import Input, merge, Convolution2D, MaxPooling2D, UpSampling2D, Flatten, Dropout
from keras.optimizers import Adam, SGD
from keras.models import Sequential, Model
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint, LearningRateScheduler
from keras.regularizers import l2
from keras import backend as K
from keras.utils import np_utils
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
import cPickle as pickle
import scipy.io
import numpy as np
import h5py

config = tf.ConfigProto()
#config.gpu_options.per_process_gpu_memory_fraction = 1.0
set_session(tf.Session(config=config))



###################################### LOADING DATA ##############################

### Create a dictionary that contains all the dictiories within each dataset (workspace) we want to load ###

#Go to the data directory
dataDir = "/scratch/barbieri/DATA_CNN_networks/"

Load_Data = {}  
for i in range(0, len(os.listdir( dataDir ))):  
    Load_Data[i] = {}  #Dictionary for a single workspace

    

### lOAD THE DATA STORING THEM IN A DICTIONARY ###

i = 0
for file in os.listdir( dataDir ):
    Load_Data[i]['File'] =  scipy.io.loadmat( dataDir+file )
    i = i+1
    

#### Define another dictionary for storing intensity curves and classes ### 
#From each dataset, the intensity curve and classes are extracted and stored in another dictionary

DATA = {}
N_of_file =len(os.listdir( dataDir ))
for i in range(0, N_of_file):
        DATA[i] = {}



#### Fill the dictionary ###

Ncell_classified = 200 #Number of classified cells per dataset
for i in range(0, N_of_file):
    #Decision array for each dataset
    DATA[i]['decision'] = Load_Data[i]['File']['decisionlist']
    #Size of the matrix to store intensity values
    NumofData = len(Load_Data[i]['File']['RAWDATA'][0,0]['files'][0,0][0]['intensity'])
    Int_curve_len = len(Load_Data[4]['File']['RAWDATA'][0,0]['files'][0,0][0]['intensity'][0][0])
    #Empty matrix to store all the intensity values of all the cells
    DATA[i]['intensity'] = np.zeros((Ncell_classified,  Int_curve_len ))
    for k in range(0,Ncell_classified): #Only the first 200 were classified
        DATA[i]['intensity'][k,:] = Load_Data[i]['File']['RAWDATA'][0,0]['files'][0,0][0]['intensity'][k][0,0:900] #copy the first 900 values

        
### Classes array: contain values from all the data ###
Classes=[]
for i in range( 0, N_of_file ):
    Classes = np.append(Classes , DATA[i]['decision']).astype(int)

    
### Intensities matrix: contain values from all the data    
NTot_Data = N_of_file*Ncell_classified
Intensity = np.zeros((0, DATA[0]['intensity'].shape[1]))
for i in range( 0, N_of_file ):
    Intensity = np.append(Intensity, DATA[i]['intensity'][:,:], axis=0)

    
### Elimate intensity profiles with NaN values and corresponding classes ###
nan_indices = np.where(np.isnan(Intensity))
Intensity = np.delete(Intensity,nan_indices[0], axis = 0)
Classes = np.delete(Classes,nan_indices[0], axis = 0)
#Redefine the total number of data
total = len(Intensity)


### Downsampling ###
data1 = Intensity[:,0:896]
step = 7
data_downsampl = data1[:,::step]
#Redefine len of the Intensity curves:
Int_profile_len = len(data_downsampl[0])


### Distribute all values randomly ###
randseq = np.random.choice(np.arange(0,total),total,replace=False)
#Data randomly organized
data_downsampl = data_downsampl[randseq,:]
Classes = Classes[randseq]

######################### load file - training data ####################
train_dt = data_downsampl[0:800,:]
train_cl = Classes[0:800]
X = train_dt  
y = train_cl 
N_train_cell = len(train_dt)
# process the data to fit in a keras CNN properly
# input data needs to be (N, C, X, Y) - shaped where
# N - number of samples
# C - number of channels per sample
# (X, Y) - sample size

X = X.reshape((N_train_cell, Int_profile_len,1, 1))
y = np_utils.to_categorical(y)


######################### load file - testing data ####################
#Define variables 
N_test_cell = total - N_train_cell
#Matrix for intensity values
test_dt = np.zeros((N_test_cell,Int_profile_len))
#Use as testing data the last N_test_cell values within the total 
test_dt[:,:]= data_downsampl[N_train_cell:total]
#Same process for the associated classes
test_cl = Classes[N_train_cell:total]

In [6]:
plot(Intensity[0])

NameError: name 'plot' is not defined